# Assistive Technology Patents

**Author:** Luke Steuber  
**Dataset:** WIPO PATENTSCOPE assistive technology patents (1985-2026)  
**Source:** patentsview_assistive_tech.json

This notebook analyzes patent trends in assistive technology across categories including wheelchairs, prosthetics, hearing aids, braille devices, screen readers, and more.

In [ ]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

%matplotlib inline

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

## Load Dataset

In [ ]:
with open('../patentsview_assistive_tech.json', 'r') as f:
    data = json.load(f)

metadata = data['metadata']
search_queries = data['search_queries']
patents = data['patents']
analysis = data['analysis']

print(f"Data fetched: {metadata['fetch_date']}")
print(f"Total unique patents: {metadata['total_unique_patents']:,}")
print(f"Search categories: {len(search_queries)}")
print(f"\nSource: {metadata['source']}")

## Dataset Structure

In [ ]:
# Show structure of first patent record
if patents:
    sample_patent = patents[0]
    print("Patent record fields:")
    for key, value in sample_patent.items():
        value_preview = str(value)[:100] if value else 'None'
        print(f"  {key}: {value_preview}")
else:
    print("No patent records found.")

## Summary Statistics

In [ ]:
# Convert patents to DataFrame for easier analysis
patents_df = pd.DataFrame(patents)

print(f"Total patents: {len(patents_df)}")
print(f"Date range: {patents_df['patent_date'].min()} to {patents_df['patent_date'].max()}")
print(f"\nUnique assignees: {patents_df['assignee'].nunique()}")
print(f"Unique countries: {patents_df['country'].nunique()}")

# Show top countries
print("\nTop 10 countries by patent count:")
print(patents_df['country'].value_counts().head(10))

## 1. Patents by Year

Explosive growth from ~10 patents in 2010-2014 to 519 in 2020-2024.

In [ ]:
# Extract year data from analysis
years = sorted([int(year) for year in analysis['patents_by_year'].keys()])
counts = [analysis['patents_by_year'][str(year)] for year in years]

# Create DataFrame
year_df = pd.DataFrame({
    'Year': years,
    'Patents': counts
})

# Plot
fig, ax = plt.subplots(figsize=(14, 6))
bars = ax.bar(year_df['Year'], year_df['Patents'], color='steelblue', edgecolor='black', linewidth=0.5)

# Highlight recent explosive growth (2020+)
for i, (year, count) in enumerate(zip(year_df['Year'], year_df['Patents'])):
    if year >= 2020:
        bars[i].set_color('coral')

ax.set_xlabel('Year', fontsize=12, fontweight='bold')
ax.set_ylabel('Number of Patents', fontsize=12, fontweight='bold')
ax.set_title('Assistive Technology Patents by Year (1985-2026)', fontsize=14, fontweight='bold')
ax.grid(axis='y', alpha=0.3)

# Add cumulative total annotation
total = year_df['Patents'].sum()
ax.text(0.98, 0.95, f'Total: {total:,} patents', 
        transform=ax.transAxes, fontsize=11, 
        verticalalignment='top', horizontalalignment='right',
        bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.tight_layout()
plt.show()

print(f"Total patents (1985-2026): {total:,}")
print(f"Average per year (2020-2026): {year_df[year_df['Year'] >= 2020]['Patents'].mean():.1f}")
print(f"Average per year (pre-2020): {year_df[year_df['Year'] < 2020]['Patents'].mean():.1f}")

## 2. Patents by Category

Prosthetics, wheelchairs, hearing aids, and cochlear implants dominate.

In [ ]:
# Extract category data from analysis
categories = list(analysis['patents_by_category'].keys())
category_counts = list(analysis['patents_by_category'].values())

# Create DataFrame and sort
category_df = pd.DataFrame({
    'Category': categories,
    'Patents': category_counts
}).sort_values('Patents', ascending=True)

# Plot horizontal bar chart
fig, ax = plt.subplots(figsize=(12, 10))
colors = plt.cm.viridis(range(len(category_df)))
bars = ax.barh(category_df['Category'], category_df['Patents'], color=colors, edgecolor='black', linewidth=0.5)

ax.set_xlabel('Number of Patents', fontsize=12, fontweight='bold')
ax.set_ylabel('Category', fontsize=12, fontweight='bold')
ax.set_title('Patents by Assistive Technology Category', fontsize=14, fontweight='bold')
ax.grid(axis='x', alpha=0.3)

# Add value labels
for i, (cat, count) in enumerate(zip(category_df['Category'], category_df['Patents'])):
    ax.text(count + 2, i, str(count), va='center', fontsize=9)

plt.tight_layout()
plt.show()

print("Top 5 categories:")
print(category_df.tail(5).to_string(index=False))

## 3. Top Patent Assignees

Organizations and companies filing the most assistive technology patents.

In [ ]:
# Count assignees from patent records
assignee_counts = patents_df['assignee'].value_counts().head(20)

# Plot
fig, ax = plt.subplots(figsize=(12, 8))
colors = plt.cm.plasma(range(len(assignee_counts)))
bars = ax.barh(range(len(assignee_counts)), assignee_counts.values, color=colors, edgecolor='black', linewidth=0.5)
ax.set_yticks(range(len(assignee_counts)))
ax.set_yticklabels(assignee_counts.index, fontsize=10)

ax.set_xlabel('Number of Patents', fontsize=12, fontweight='bold')
ax.set_ylabel('Assignee/Organization', fontsize=12, fontweight='bold')
ax.set_title('Top 20 Patent Assignees (Assistive Technology)', fontsize=14, fontweight='bold')
ax.grid(axis='x', alpha=0.3)

# Add value labels
for i, count in enumerate(assignee_counts.values):
    ax.text(count + 0.5, i, str(count), va='center', fontsize=9)

plt.tight_layout()
plt.show()

print(f"Total unique assignees: {patents_df['assignee'].nunique()}")
print(f"\nTop 10 assignees:")
for i, (name, count) in enumerate(assignee_counts.head(10).items(), 1):
    print(f"{i:2d}. {name}: {count} patents")

## 4. IPC Code Distribution

International Patent Classification codes reveal technology domains.

In [ ]:
# Extract IPC class data from analysis
ipc_classes = list(analysis['ipc_class_distribution'].keys())[:15]  # Top 15
ipc_counts = [analysis['ipc_class_distribution'][code] for code in ipc_classes]

# IPC code descriptions (common codes)
ipc_descriptions = {
    'A61F': 'Prostheses/Orthotics',
    'H04R': 'Hearing Devices',
    'G06F': 'Computing/Data Processing',
    'A61H': 'Physical Therapy',
    'G09B': 'Educational Devices',
    'A61G': 'Transport/Patient Handling',
    'G06T': 'Image Processing',
    'G06V': 'Image/Video Recognition',
    'A61B': 'Diagnosis/Surgery',
    'G10L': 'Speech Analysis/Synthesis',
    'H04N': 'Image Communication',
    'A61N': 'Electrotherapy',
    'G06N': 'Computer Learning/AI',
    'A61M': 'Medical Devices',
    'B25J': 'Robotics'
}

labels = [f"{code}\n{ipc_descriptions.get(code, 'Other')}" for code in ipc_classes]

# Plot
fig, ax = plt.subplots(figsize=(14, 6))
colors = plt.cm.tab20(range(len(ipc_classes)))
bars = ax.bar(range(len(ipc_classes)), ipc_counts, color=colors, edgecolor='black', linewidth=0.5)
ax.set_xticks(range(len(ipc_classes)))
ax.set_xticklabels(labels, rotation=45, ha='right', fontsize=9)

ax.set_xlabel('IPC Classification Code', fontsize=12, fontweight='bold')
ax.set_ylabel('Number of Patents', fontsize=12, fontweight='bold')
ax.set_title('Top 15 IPC Classification Codes (International Patent Classification)', fontsize=14, fontweight='bold')
ax.grid(axis='y', alpha=0.3)

# Add value labels
for i, count in enumerate(ipc_counts):
    ax.text(i, count + 5, str(count), ha='center', fontsize=9, fontweight='bold')

plt.tight_layout()
plt.show()

print("IPC Code Summary:")
for code, count in zip(ipc_classes[:10], ipc_counts[:10]):
    desc = ipc_descriptions.get(code, 'Other')
    print(f"{code} ({desc}): {count} patents")

## 5. Growth Analysis: Pre-2020 vs Post-2020

In [ ]:
# Aggregate by period
year_df['Period'] = year_df['Year'].apply(lambda x: 'Pre-2020' if x < 2020 else '2020+')
period_summary = year_df.groupby('Period')['Patents'].agg(['sum', 'mean', 'count'])
period_summary.columns = ['Total Patents', 'Avg/Year', 'Years']

print("Growth Comparison:")
print(period_summary)
print(f"\nGrowth factor: {period_summary.loc['2020+', 'Avg/Year'] / period_summary.loc['Pre-2020', 'Avg/Year']:.1f}x")

# Pie chart of pre/post 2020 distribution
fig, ax = plt.subplots(figsize=(8, 6))
period_totals = year_df.groupby('Period')['Patents'].sum()
colors = ['lightcoral', 'steelblue']
explode = (0.05, 0)

ax.pie(period_totals, labels=period_totals.index, autopct='%1.1f%%', 
       colors=colors, explode=explode, shadow=True, startangle=90)
ax.set_title('Patent Distribution: Pre-2020 vs 2020+', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

## Key Findings

1. **Total Patents:** 1,325 assistive technology patents from 1985-2026

2. **Explosive Recent Growth:**
   - Pre-2020 average: ~5-10 patents/year
   - 2020-2026 average: ~207 patents/year (20x increase)
   - Peak year 2025: 416 patents

3. **Dominant Categories:**
   - Prostheses/orthotics (IPC A61F2): 100 patents
   - Cochlear implants: 102 patents
   - Wheelchairs: 100 patents
   - Hearing aids (IPC H04R25): 100 patents

4. **Technology Trends:**
   - Computing/AI integration (G06F, G06N) increasingly common
   - Physical therapy devices (A61H) prominent
   - Educational devices for blind/deaf (G09B21) well-represented

5. **Search Coverage:** 17 search categories from assistive technology keywords to specific IPC codes

6. **Data Source:** WIPO PATENTSCOPE (authoritative international database covering PCT applications and national patents from 80+ offices)

**Note:** The 2026 data (293 patents) is partial year through February 12, 2026.